In [13]:
import torch
import os
import csv
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
from transformers import CLIPTokenizer


In [14]:
from ldm.dpm.ddpm import DDPM
from ldm.vae.encoder import Encoder
from ldm.vae.decoder import Decoder
from  ldm.clip.cliper import CLIP

In [15]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, csv_file):
        self.root_dir = root_dir
        self.csv_file = csv_file
        gen=torch.Generator()
        self.diff=DDPM(gen)
        self.tokenizer = CLIPTokenizer("utils/vocab.json", merges_file="utils/merges.txt")

        self.txt_encoder=CLIP()
        self.txt.load
        self.time_steps=self.diff.timesteps
        self.encoder=Encoder()
        self.transform= transforms.Compose([
            transforms.Lambda(lambda t: (t/255)),
            transforms.Lambda(lambda t: (t * 2) - 1),
            transforms.Lambda(lambda t: t.permute([2,0,1])), # Scale data between [-1, 1] 
])



        with open(self.csv_file, "r") as f:
            reader = csv.reader(f)
            self.image_paths = []
            self.txt=[]
            
            for row in reader:
                image,txt = row[0], row[2]
                self.image_paths.append(os.path.join(self.root_dir,image))
                self.txt.append(txt)
                
                
              

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = self.image_paths[index]
        
        t = torch.randint(0, self.time_steps,(1,))

        image = torch.from_numpy(np.array((Image.open(image))))

        with torch.no_grad():
        
            image = self.transform(image)
            noisy_image=self.diff.add_noise(image,t)
            noisy_image=self.encoder(noisy_image)
            text_guiance=self.txt_encoder(self.tokenizer.batch_encode_plus([self.txt[index]],padding="max_length", max_length=77).input_ids)
            

        
   

        return image,self.encoder(noisy_image),t,text_guiance




In [6]:
tokenizer = CLIPTokenizer("utils/vocab.json", merges_file="utils/merges.txt")

In [8]:
with torch.no_grad():
    bloc1=Encoder()
    #bloc1=torch.load('data/encoder_params.pt')
    v=torch.randn(1,3,512,512)
    v=bloc1(v)
    v.shape

In [10]:
prompt="the quick brown fox jumps over the lazy hound"

In [11]:
token_prompt=tokenizer.batch_encode_plus(
                [prompt], padding="max_length", max_length=77
            ).input_ids